In [81]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectFromModel, SelectKBest, chi2
from sklearn.linear_model import Lasso
# import xgboost as xgb
#DecisionTreeClassifier
from sklearn.tree import DecisionTreeClassifier,DecisionTreeRegressor
from typing import List, Dict,Tuple,Any
from scipy.stats import chi2_contingency
import scipy.stats as stats

# Pipeline imports
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

#import LogisticRegression
from sklearn.linear_model import LogisticRegression
#import RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.base import TransformerMixin
#impory accuracy_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

import pickle

from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectFromModel, SelectKBest, chi2
from sklearn.model_selection import train_test_split

#ignore warnings
import warnings
warnings.filterwarnings('ignore')


ModuleNotFoundError: No module named 'sklearn'

In [ ]:
#list names of all files
os.listdir('data')


In [ ]:
#read the data
list_of_files = os.listdir('data')
leads, air, land, sea = [pd.read_csv('data/' + file) for file in list_of_files]

In [ ]:
leads


In [ ]:
air


In [ ]:
rates_air_renamed = air.rename(columns={
    'origin_country': 'port_from_country',
    'destination_country': 'port_to_country',
    'load_type': 'container_type'
})

# Объединение данных о запросах с тарифами для авиаперевозок.
# В реальной ситуации потребуется учитывать периоды действия тарифов, что может потребовать дополнительной обработки.
merged_leads_air = pd.merge(
    leads,
    rates_air_renamed,
    how='inner',
    on=['port_from_country', 'port_to_country', 'container_type']
)

merged_leads_air.head()


In [ ]:
merged_leads_air['price_usd']

In [ ]:
# Подготовка и объединение данных о запросах на перевозку с тарифами для наземных перевозок.
rates_land_renamed = land.rename(columns={
    'origin_country': 'port_from_country',
    'destination_country': 'port_to_country',
    'load_type': 'container_type'
})
merged_leads_land = pd.merge(
    leads,
    rates_land_renamed,
    how='left',
    on=['port_from_country', 'port_to_country', 'container_type']
)

# Подготовка и объединение данных о запросах на перевозку с тарифами для морских перевозок.
rates_sea_renamed = sea.rename(columns={
    'origin_country': 'port_from_country',
    'destination_country': 'port_to_country',
    'load_type': 'container_type'
})
merged_leads_sea = pd.merge(
    leads,
    rates_sea_renamed,
    how='left',
    on=['port_from_country', 'port_to_country', 'container_type']
)



In [ ]:
merged_leads_land['price_usd'].describe()


In [ ]:
# merged_leads_land анализ
#в основном цены только на leads_land

In [ ]:
merged_leads_land['price_usd'].describe()

In [ ]:
merged_leads_land.info()

In [ ]:
#sepatare data for null and not null
merged_leads_land_null = merged_leads_land[merged_leads_land['price_usd'].isnull()]
merged_leads_land_not_null = merged_leads_land[merged_leads_land['price_usd'].notnull()]


In [ ]:
merged_leads_land_not_null